In [1]:
# Importando bibliotecas
import pandas as pd
from datetime import datetime # Vou usar essa biblioteca para criar coluna 'idade'

In [2]:
# Importando arquivos JÁ TRATADOS anteriormente

df_clientes_novo = pd.read_excel('clientes_novo.xlsx')
df_transacoes_novo = pd.read_excel('transacoes_novo.xlsx')
df_campanhas_novo = pd.read_excel('campanhas_novo.xlsx')

In [3]:
# Quero criar uma coluna 'idade' na base clientes
df_clientes_novo['idade'] = datetime.now().year - df_clientes_novo['data_nascimento'].dt.year

# Quero criar também uma coluna 'tempo_cooperado' na base clientes
df_clientes_novo['tempo_cooperado'] = datetime.now().year - df_clientes_novo['data_adesao'].dt.year

In [4]:
# Agora quero criar uma base geral CLIENTES + TRANSAÇÕES para começar a responder as perguntas
df_cli_transacoes = pd.merge(df_transacoes_novo, df_clientes_novo, on='id_cooperado', how='inner')

In [5]:
# A base unificada está pronta, mas não posso me basear nesses números, pois um id de cooperado pode ter feito mais de uma transação
# Dessa forma, agora irei agrupar os valores das transações pelo id do cooperado
# Além disso, quero uma contagem de quantas transações cada id_cooperado fez

df_valores = df_cli_transacoes.groupby('id_cooperado', as_index=False).agg({
    'valor': 'sum',
    'id_transacao': 'count'
})


# Agora quero renomear a coluna, para que sugira que é uma contagem de transações
df_valores.rename(columns={'id_transacao': 'count_transacoes'}, inplace=True)

In [6]:
# Agora quero puxar informações do cooperado (base clientes)
# Pra isso vou fazer um left join, ou seja, pegar o valor somado de todas as transações por id + infos dos clientes
df_valor_clientes = pd.merge(df_valores, df_clientes_novo, on='id_cooperado', how='left')

# Agora a minha base df_valor_clientes vai me auxiliar a responder várias perguntas

In [23]:
# PERGUNTAS

# Pergunta 1: Perfil demográfico -- IDADE MÉDIA
idade_min = df_valor_clientes['idade'].min()
idade_max = df_valor_clientes['idade'].max()
idade_media = df_valor_clientes['idade'].mean()

# Printar na tela os valores
print(f'Cooperado usuário de cartão mais jovem -> {idade_min} anos.')
print(f'Cooperado usuário de cartão mais velho -> {idade_max} anos.')
print(f'Idade média do cooeprado usuário de cartão -> {idade_media:,.2f} anos.')

Cooperado usuário de cartão mais jovem -> 23 anos.
Cooperado usuário de cartão mais velho -> 51 anos.
Idade média do cooeprado usuário de cartão -> 37.27 anos.


In [9]:
# PERGUNTAS

# Pergunta 1: Perfil demográfico -- POR GÊNERO - TEMPO MEDIO COOPERADO - IDADE MÉDIA - RENDA_ESTIMADA

df_agrupado_genero = df_valor_clientes.groupby('genero').agg({
    'valor': 'sum',
    'count_transacoes': 'sum',
    'renda_estimada': 'mean',
    'tempo_cooperado': 'mean',
    'idade': 'mean'
}).rename(columns={
    'valor': 'valor_total',
    'count_transacoes': 'qtde_transacoes',
    'renda_estimada': 'renda_media',
    'tempo_cooperado': 'tempo_medio_cooperado',
    'idade': 'idade_media'
})

df_agrupado_genero['valor_medio_transacao'] = df_agrupado_genero['valor_total'] / df_agrupado_genero['qtde_transacoes']
df_agrupado_genero

,valor_total,qtde_transacoes,renda_media,tempo_medio_cooperado,idade_media,valor_medio_transacao
genero,,,,,,
feminino,1837.75,13,4472.977273,6.363636,31.090909,141.365385
masculino,3604.50,16,8136.363636,13.000000,43.454545,225.281250


In [10]:
# Pergunta 1: Perfil demográfico -- PRINCIPAIS ESTADOS

df_agrupado_estado = df_valor_clientes.groupby('estado').agg({
    'valor': 'sum',
    'count_transacoes': 'sum',
    'renda_estimada': 'mean',
    'tempo_cooperado': 'mean',
}).rename(columns={
    'valor': 'valor_total',
    'count_transacoes': 'qtde_transacoes',
    'renda_estimada': 'renda_media',
    'tempo_cooperado': 'tempo_medio_cooperado',
})

df_agrupado_estado['valor_medio_transacao'] = df_agrupado_estado['valor_total'] / df_agrupado_estado['qtde_transacoes']
df_agrupado_estado.sort_values(by='qtde_transacoes', ascending=False)

,valor_total,qtde_transacoes,renda_media,tempo_medio_cooperado,valor_medio_transacao
estado,,,,,
SC,1287.90,9,6483.500000,10.500000,143.100000
SP,1109.20,7,6200.100000,7.400000,158.457143
RJ,504.65,6,4375.125000,6.750000,84.108333
MG,1715.50,3,9366.916667,12.333333,571.833333
PR,615.00,2,8650.000000,12.500000,307.500000
RS,210.00,2,2950.000000,12.000000,105.000000


In [11]:
# Pergunta 1: Perfil demográfico -- PRINCIPAIS CIDADES

df_agrupado_cidade = df_valor_clientes.groupby('cidade').agg({
    'valor': 'sum',
    'count_transacoes': 'sum',
    'renda_estimada': 'mean',
    'tempo_cooperado': 'mean',
}).rename(columns={
    'valor': 'valor_total',
    'count_transacoes': 'qtde_transacoes',
    'renda_estimada': 'renda_media',
    'tempo_cooperado': 'tempo_medio_cooperado',
})

df_agrupado_cidade['valor_medio_transacao'] = df_agrupado_cidade['valor_total'] / df_agrupado_cidade['qtde_transacoes']
df_agrupado_cidade.sort_values(by='qtde_transacoes', ascending=False)

,valor_total,qtde_transacoes,renda_media,tempo_medio_cooperado,valor_medio_transacao
cidade,,,,,
blumenau,901.90,7,6900.050000,11.500000,128.842857
sao_paulo,1109.20,7,6200.100000,7.400000,158.457143
rio_de_janeiro,504.65,6,4375.125000,6.750000,84.108333
belo_horizonte,1715.50,3,9366.916667,12.333333,571.833333
curitiba,615.00,2,8650.000000,12.500000,307.500000
porto_alegre,210.00,2,2950.000000,12.000000,105.000000
florianopolis,386.00,2,5650.400000,8.500000,193.000000


In [12]:
# Pergunta 2: Perfil dos estabelecimentos

# Agora eu volto lá naquele dataframe do início -> df_cli_transacoes
# A intenção é agrupar valores pelo segmento dos estabelecimentos que nossos cooperados usam o cartão

df_valores_cat_estabelecimento = df_cli_transacoes.groupby('categoria_estabelecimento', as_index=False).agg({
    'valor': 'sum',
    'id_transacao': 'count'
})


# Agora quero renomear a coluna, para que sugira que é uma contagem de transações
df_valores_cat_estabelecimento.rename(columns={'id_transacao': 'count_transacoes'}, inplace=True)
df_valores_cat_estabelecimento['valor_medio_transacao'] = df_valores_cat_estabelecimento['valor'] / df_valores_cat_estabelecimento['count_transacoes']
df_valores_cat_estabelecimento.sort_values(by='count_transacoes', ascending=False)


,categoria_estabelecimento,valor,count_transacoes,valor_medio_transacao
5,restaurante,483.80,6,80.633333
7,supermercado,1292.55,6,215.425000
1,combustível,755.00,4,188.750000
3,farmácia,223.40,3,74.466667
9,vestuário,490.00,2,245.000000
6,serviços,115.00,2,57.500000
4,lazer,262.30,2,131.150000
2,eletrônicos,1250.00,1,1250.000000
0,alimentação,45.00,1,45.000000
8,transporte,75.20,1,75.200000


In [13]:
# NOVA ANÁLISE -- CAMPANHAS

# Trazer para a base geral os valores da base CAMPANHAS
# Qual a intenção aqui? Ver o comportamento dos cooperados que foram alcançados por nossas campanhas
# Pra isso vou trazer para a base de campanhas o valor que cada id_cooperado usou em transações
# Sei que a informação de valor da transação vai ficar duplicada.
# Pois várias interações podem ocorrer para um mesmo usuário.

df_campanhas_transacoes = pd.merge(df_campanhas_novo, df_valor_clientes, on='id_cooperado', how='left')
# df_campanhas_transacoes.sort_values(by='id_cooperado', ascending=False)
df_campanhas_transacoes[df_campanhas_transacoes['acao']=='abriu']

,id_interacao,id_cooperado,nome_campanha,canal,data_interacao,acao,valor,count_transacoes,nome,data_nascimento,genero,cidade,estado,data_adesao,renda_estimada,idade,tempo_cooperado
0,1001,101,cashback_fim_de_ano,email,2024-12-01,abriu,325.5,3.0,Joao Silva,1985-03-15,masculino,sao_paulo,SP,2015-01-20,5500.00,40.0,10.0
3,1004,105,investimento_facilitado,email,2024-12-10,abriu,245.8,2.0,Pedro Martins,1988-06-10,masculino,blumenau,SC,2016-11-25,6200.00,37.0,9.0
10,1011,107,investimento_facilitado,email,2024-12-22,abriu,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaT,NaN,NaN,NaN
12,1013,115,crédito_consignado,sms,2024-11-28,abriu,450.0,1.0,Fernando Gomes,1979-09-21,masculino,curitiba,PR,2013-01-30,8200.00,46.0,12.0
13,1014,104,cashback_fim_de_ano,email,2024-12-03,abriu,45.0,1.0,Ana Costa,1995-01-30,feminino,belo_horizonte,MG,2020-08-01,3100.75,30.0,5.0
18,1019,125,crédito_consignado,email,2024-12-04,abriu,150.0,1.0,Vinicius Moraes,1990-02-14,masculino,porto_alegre,RS,2017-03-25,6400.00,35.0,8.0
23,1024,106,investimento_facilitado,sms,2024-12-13,abriu,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaT,NaN,NaN,NaN
27,1028,109,seguro_premiado,social_media,2024-12-17,abriu,60.0,1.0,Rafael Souza,1975-02-28,masculino,porto_alegre,RS,2009-04-05,-500.00,50.0,16.0
31,1032,119,natal_premiado,email,2024-11-27,abriu,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaT,NaN,NaN,NaN
34,1035,131,cashback_fim_de_ano,social_media,2024-12-18,abriu,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaT,NaN,NaN,NaN


In [14]:
# Aqui quero ver o perfil dos usuários por canal de campanha

df_campanhas_transacoes_canais = df_campanhas_transacoes.groupby('canal').agg({
    'valor': 'sum',
    'count_transacoes': 'sum',
    'tempo_cooperado': 'mean',
}).rename(columns={
    'valor': 'valor_total',
    'count_transacoes': 'qtde_transacoes',
    'tempo_cooperado': 'tempo_medio_cooperado',
})

df_campanhas_transacoes_canais['valor_medio_transacao'] = df_campanhas_transacoes_canais['valor_total'] / df_campanhas_transacoes_canais['qtde_transacoes']
df_campanhas_transacoes_canais.sort_values(by='qtde_transacoes', ascending=False)

,valor_total,qtde_transacoes,tempo_medio_cooperado,valor_medio_transacao
canal,,,,
sms,2518.10,16.0,9.777778,157.381250
email,2839.30,14.0,9.333333,202.807143
push_app,1306.90,11.0,10.857143,118.809091
social_media,365.75,3.0,9.000000,121.916667


In [21]:
# Aqui vamos olhar o canal, ação e nome da campanha

df_campanhas_acao = df_campanhas_transacoes.groupby(['canal', 'acao', 'nome_campanha']).agg({
    'id_cooperado': 'nunique',
    'valor':'sum',
    'idade': 'mean'
}).reset_index().rename(columns={
    'id_cooperado': 'qtde_cooperados_impactados',
    'valor': 'valor_transacoes',
    'idade': 'idade_media'
}).sort_values(by='qtde_cooperados_impactados', ascending=False)
df_campanhas_acao[df_campanhas_acao['acao']=='converteu']


,canal,acao,nome_campanha,qtde_cooperados_impactados,valor_transacoes,idade_media
7,email,converteu,investimento_facilitado,1,98.7,49.0
6,email,converteu,cashback_fim_de_ano,1,0.0,NaN
11,push_app,converteu,natal_premiado,1,68.9,36.0


In [16]:
# Aqui eu quero entender se há relação do número de transações com o nome da campanha
# Mesmo eu sabendo que a campanha foi enviada em período diferente das transações.

df_campanhas_acao_perfil = df_campanhas_transacoes.groupby(['nome_campanha', 'acao']).agg({
    'id_cooperado': 'nunique',
    'count_transacoes': 'sum',
    'valor':'sum',
    'idade': 'mean'
}).reset_index().rename(columns={
    'id_cooperado': 'qtde_cooperados_impactados',
    'count_transacoes': 'qtde_transacoes',
    'valor': 'valor_transacoes',
    'idade': 'idade_media'
}).sort_values(by='qtde_cooperados_impactados', ascending=False)

df_campanhas_acao_perfil

,nome_campanha,acao,qtde_cooperados_impactados,qtde_transacoes,valor_transacoes,idade_media
1,cashback_fim_de_ano,clicou,4,9.0,1023.10,38.250000
0,cashback_fim_de_ano,abriu,3,4.0,370.50,35.000000
3,cashback_fim_de_ano,visualizou,3,5.0,768.80,33.333333
7,investimento_facilitado,abriu,3,2.0,245.80,37.000000
16,seguro_premiado,clicou,3,7.0,784.00,30.000000
6,crédito_consignado,visualizou,2,1.0,165.00,40.000000
17,seguro_premiado,visualizou,2,2.0,490.80,33.000000
14,natal_premiado,visualizou,2,3.0,570.90,42.333333
10,investimento_facilitado,visualizou,2,4.0,421.25,33.500000
4,crédito_consignado,abriu,2,2.0,600.00,40.500000


In [17]:
# Aqui eu quero entender tudo sobre o cooperado e o vínculo com as campanhas

df_clientes_campanhas = pd.merge(df_clientes_novo, df_campanhas_novo, on='id_cooperado', how='left')
df_clientes_campanhas['nome_campanha'] = df_clientes_campanhas['nome_campanha'].fillna('sem_campanha')
# df_clientes_campanhas.sort_values(by='renda_estimada', ascending=False)


In [18]:
# Agora quero criar um df_geral para armazenar todas as informações
df_total_geral = pd.merge(df_clientes_campanhas, df_transacoes_novo, on='id_cooperado', how='left')

# Quero ver onde tem mais transações nas cidades, em quais tipos de estabelecimentos
df_agrupado = df_total_geral.groupby(['cidade', 'categoria_estabelecimento'])['id_transacao'].count().reset_index()
df_agrupado.sort_values(by=['cidade', 'id_transacao'], ascending=[True, False])


,cidade,categoria_estabelecimento,id_transacao
0,belo_horizonte,alimentação,1
1,belo_horizonte,eletrônicos,1
2,belo_horizonte,supermercado,1
3,blumenau,combustível,5
6,blumenau,restaurante,4
4,blumenau,farmácia,2
5,blumenau,lazer,1
7,curitiba,combustível,1
8,curitiba,viagem,1
10,florianopolis,transporte,2


In [19]:
# Agora eu quero ver as cidades com mais interações das campanhas

df_ag_campanhas = pd.merge(df_campanhas_novo, df_clientes_novo, on='id_cooperado', how='left')
df_ag_campanhas = df_ag_campanhas.groupby(['cidade', 'nome_campanha'])['id_interacao'].count().reset_index()
df_ag_campanhas

,cidade,nome_campanha,id_interacao
0,belo_horizonte,cashback_fim_de_ano,1
1,belo_horizonte,natal_premiado,2
2,blumenau,cashback_fim_de_ano,3
3,blumenau,crédito_consignado,1
4,blumenau,investimento_facilitado,1
5,blumenau,natal_premiado,1
6,blumenau,seguro_premiado,1
7,curitiba,crédito_consignado,3
8,curitiba,investimento_facilitado,1
9,florianopolis,natal_premiado,2


In [20]:
# Ver informações de renda por idade
bins = [19, 29, 39, 59]
labels = ['20-29', '30-39', '40-59']
df_clientes_campanhas['faixa_idade'] = pd.cut(df_clientes_campanhas['idade'], bins=bins, labels=labels)

df_clientes_campanhas.groupby('faixa_idade')['renda_estimada'].mean().reset_index(name='media_renda')

C:\Users\rpani\AppData\Local\Temp\ipykernel_11436\368936280.py:6: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  df_clientes_campanhas.groupby('faixa_idade')['renda_estimada'].mean().reset_index(name='media_renda')


,faixa_idade,media_renda
0,20-29,3520.050000
1,30-39,5390.906250
2,40-59,8741.176471
